# MADDPG - Collaboration and Competition
---

Say something

In [1]:
from collections import deque
import numpy as np
import torch

from models.actor import Actor
from models.critic import Critic
from agent import Agent

from unityagents import UnityEnvironment
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
path = 'env/Tennis.app'

env = UnityEnvironment(file_name=path)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Getting the `brain_name`, a **brain** is responsible for deciding the actions of their associated agents. 

In [3]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

## Checking the State and Action spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal is to keep the ball in play.

The observation space consists of 24 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

In [4]:
env_info = env.reset(train_mode=True)[brain_name]

num_agents = len(env_info.agents)
print(f'Number of agents: {num_agents}')

action_size = brain.vector_action_space_size
print(f'Size of each action: {action_size}')

states = env_info.vector_observations
state_size = states.shape[1]
print(f'There are {num_agents} agents. Each observes a state with length: {state_size}')
print(f'The state for the first agent looks like: {states[0]}')

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


## Random Actions

Let's take random actions on this environment, to watch the kind of rewards our agents would get from this.

In [5]:
for i in range(1, 6):
    env_info = env.reset(train_mode=False)[brain_name]
    states = env_info.vector_observations
    scores = np.zeros(num_agents)
    
    while True:
        actions = np.random.randn(num_agents, action_size)
        actions = np.clip(actions, -1, 1)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        
        scores += env_info.rewards
        states = next_states
        
        if np.any(dones):
            break
            
    print(f'Score (max over agents) from episode {i}: {np.max(scores)}')

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0


KeyboardInterrupt: 

In [ ]:
env.close()

## Solving the Environment -- MADDPG

This environment will be solved using MADDPG, which is the multi-agent version of the DDPG algorithm. DDPG is an algorithm based on [Deep Q-Networks](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) to work on continous problems. It works with two networks: an actor and a critic. The actor is responsible for, given a state, perform an action (deterministic algorithm, choosing a single action, and not probabilities over the possible actions as DQN), while the critic will predict a score for this action on the state (judging how good was the action performed by the actor). 

Like DQNs, DDPG also works with Experience Replay, where it stores a bunch of `(state, action, rewards, next_state, done)` tuples and then sample it randomly for training (removing the correlation between state-action tuples). 

Also like DQNs, the DDPG has a `target` network for the actor and the critic, responsible for predicting the targets on the training stage. The weights of the local networks (the ones being trained) are used to *soft update* the weights of the `target` network, it means, after each iteration, a the weights of the networks are "*mixed*" together based on a parameter τ (which is much less than 1, we have used 0.001). The resulting weights of the `target` network are composed of τ times the weights of the `local` plus (1 - τ) times the current weights of the `target`. This technique has been proved to help stability during learning, ahd improves the result of the algorithm (it could also be used on the DQN arquitecture).

Deep reinforcement learning algorithms are successful because of a duality of exploration and exploitation. On the beginning of the training, the agent does not know much about the environment, and to help it explore the environment correctly, it is usual to add some randomness to the action-choosing process. On the DQN paper, this was implementeed based on a ε-greedy technique, where the agent could choose both the best known action or a random one (on the beginning of the training, the agent would usually choose random actions, and as training develops, the agent gets more *trustyworthy* and starts to choose the best known action). Since the DDPG does not work with probabilities over actions, it cannot make use of ε-greedy for exploration, and to address this problem it's often used to add some noise on the actions on the beginning of training (on the DDPG paper they've used Orsntein-Uhlenbeck noise process, on this work I've used the normal distribution). The noise added to the action keeps getting smaller as the number of episodes get bigger, and later on training the agent is always choosing the predicted action (without any noise). 

The DDPG algorithm used here has some differences than the original implementation, mainly because of the *multiple agent* environment. To make use of the distributed training, the algorithm records the experiences from all the agents (adding them to the Replay Buffer), and then every **20 timesteps** the agents are updated we use this experiences to **train the agents for 10 times**. To stabilize training, I've also clipped the *Critic's Network* gradients, to avoid rapidly changes and to help convergence. To address the issue of exploration, I've used the *Normal Distribution* as a noise function, that will be multiplied by a ε parameter, that will be smaller over time, making the actor chooses less randomness over the course of training. 

In [5]:
agent = Agent(state_size, action_size, 25, [400, 300], [400, 300])
n_episodes = 5000
max_t = 3000
print_every = 100
train = True

eps_start = 2.0
eps_decay = 0.999
eps_min = 0.01

In [ ]:
scores_deque = deque(maxlen=100)
all_scores = []

eps = eps_start

print('Start training session...\n')

for i_episode in range(1, n_episodes+1):
    env_info = env.reset(train_mode=train)[brain_name]
    states = env_info.vector_observations
    agent.reset()
    scores = np.zeros(num_agents)
    for t in range(1, max_t+1):
        actions = agent.act(states, add_noise=train, epsilon=eps)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        scores += env_info.rewards
        
        if train:
            agent.step(states, actions, rewards, next_states, dones, t)
        states = next_states
        
        if np.any(dones):
            break
    eps = max(eps_min, eps * eps_decay)
    scores_deque.append(np.max(scores))
    all_scores.append(scores)
    
    if i_episode % print_every == 0 or np.mean(scores_deque) >= 0.5:
        print(f'Episode {i_episode}: {np.mean(scores_deque):.4f}')
        torch.save(agent.actor_local.state_dict(), 'model_parameters/checkpoint_actor.pth')
        torch.save(agent.actor_local.state_dict(), 'model_parameters/checkpoint_critic.pth')
        
        if np.mean(scores_deque) >= 0.5:
            print(f'Environment solved in episode {i_episode}')
            break

Start training session...

Episode 100: 0.0000


## Tests

In [7]:
import torch.nn.functional as F

In [8]:
agent = Agent(state_size, action_size, 25, [400, 300], [400, 300])

In [9]:
env_info = env.reset(train_mode=True)[brain_name]

In [10]:
states = env_info.vector_observations

In [11]:
states.shape

(2, 24)

In [12]:
actions = agent.act(states)

In [13]:
actions.shape

(2, 2)

In [14]:
states_tensor = torch.from_numpy(states).float()

f1 = agent.critic_local.fc1(states_tensor)
f1 = agent.critic_local.bn1(f1)
f1 = F.relu(f1)

In [15]:
f1.shape

torch.Size([2, 400])

In [16]:
actions_tensor = torch.from_numpy(actions).float()

f2 = torch.cat([f1, actions_tensor], dim=1)

In [17]:
f2.shape

torch.Size([2, 402])

In [18]:
agent.critic_local

Critic(
  (fc1): Linear(in_features=24, out_features=400, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=402, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=1, bias=True)
)

In [21]:
env.step(actions)[brain_name].rewards

[0.0, 0.0]